In [40]:
from keras.utils import image_dataset_from_directory
from keras.layers import Input,Conv2D,MaxPooling2D,Dropout,GlobalAveragePooling2D,Dense
from keras.models import Model
from keras import backend as k


In [6]:
data = image_dataset_from_directory(
    "images",
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=1,
    image_size=(64,64),
    shuffle=False,
    seed=69
)

Found 400 files belonging to 40 classes.


In [13]:
def siamese(inputs):
    x = Conv2D(96, (11, 11), padding="same", activation="relu")(inputs)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(256, (5, 5), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    x = Conv2D(384, (3, 3), padding="same", activation="relu")(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.3)(x)

    pooledOutput = GlobalAveragePooling2D()(x)
    pooledOutput = Dense(1024)(pooledOutput)
    outputs = Dense(128)(pooledOutput)
    return outputs

In [50]:
def euclidean_distance(vectors):
    (featA, featB) = vectors
    sum_squared = k.sum(k.square(featA - featB), axis=1, keepdims=True)
    return k.sqrt(k.maximum(sum_squared, k.epsilon()))

In [47]:
input_img = Input(shape=(64, 64, 3))
feature_extractor = Model(input_img, siamese(input_img))

In [48]:
for i, img in enumerate(data.as_numpy_iterator()):
    if i == 0:
        imgA = img[0]
    if i == 1:
        imgB = img[0]


In [49]:
encodingA = feature_extractor(imgA)
encodingB = feature_extractor(imgB)

In [51]:
distance = euclidean_distance((encodingA, encodingB))
outputs = Dense(1, activation="sigmoid")(distance)

In [52]:
outputs

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.463331e-07]], dtype=float32)>